In [1]:
import numpy as np
import pandas as pd

In [2]:
submit=pd.read_csv('/kaggle/input/daconcard/open/sample_submission.csv')
train=pd.read_csv('/kaggle/input/daconcard/open/train.csv')
test=pd.read_csv('/kaggle/input/daconcard/open/test.csv')

In [3]:
!pip install pycaret

     |████████████████████████████████| 261 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 3.8 MB/s ta 0:00:01
     |████████████████████████████████| 6.8 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 14.2 MB 13.7 MB/s eta 0:00:01    |████                            | 1.8 MB 13.7 MB/s eta 0:00:01     |███████████▉                    | 5.3 MB 13.7 MB/s eta 0:00:01     |███████████████████▊            | 8.7 MB 13.7 MB/s eta 0:00:01     |██████████████████████          | 9.7 MB 13.7 MB/s eta 0:00:01     |████████████████████████        | 10.6 MB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 167 kB 37.5 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 2.3 MB/s  eta 0:00:01
     |████████████████████████████████| 370 kB 40.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 34.0 MB/s eta 0:00:01
     |████████████████████████████████| 9.9 MB 35.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.7

In [4]:
from pycaret.classification import *

clf=setup(session_id=71,data=train,target='credit',train_size=0.8,use_gpu=True)

,Description,Value
0,session_id,71
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 20)"
5,Missing Values,True
6,Numeric Features,6
7,Categorical Features,13
8,Ordinal Features,False
9,High Cardinality Features,False


In [5]:
from sklearn.metrics import log_loss
add_metric('logloss','LogLoss',log_loss,greater_is_better=False,target='pred_proba')

# best=compare_models(sort='logloss',exclude=['svm','ridge'])

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7fee091d99e0>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [6]:
catboost=create_model('catboost')
tune_cat=tune_model(catboost,optimize='logloss')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.7029,0.7211,0.4636,0.6740,0.6514,0.2906,0.3345,0.7587
1,0.7024,0.7059,0.4556,0.6852,0.6452,0.2790,0.3304,0.7713
2,0.7114,0.7129,0.4648,0.6841,0.6561,0.3097,0.3571,0.7663
3,0.7062,0.7266,0.4609,0.6879,0.6498,0.2861,0.3426,0.7460
4,0.7123,0.7291,0.4688,0.6994,0.6578,0.3062,0.3596,0.7478
5,0.7046,0.7127,0.4506,0.6726,0.6447,0.2810,0.3352,0.7639
6,0.7240,0.7405,0.4803,0.7079,0.6715,0.3349,0.3920,0.7365
7,0.7108,0.7238,0.4716,0.6863,0.6600,0.3105,0.3554,0.7498
8,0.6985,0.7055,0.4514,0.6731,0.6414,0.2668,0.3181,0.7761
9,0.6961,0.7117,0.4497,0.6595,0.6403,0.2684,0.3137,0.7705


In [7]:
final_cat=finalize_model(tune_cat)

In [9]:
prep_pipe=get_config("prep_pipe")
prep_pipe.steps.append(['trained_model',final_cat])
pred=prep_pipe.predict_proba(test)

In [10]:
submit['0']=pred[:,0]
submit['1']=pred[:,1]
submit['2']=pred[:,2]

submit.head()

,index,0,1,2
0,26457,0.042933,0.053644,0.903423
1,26458,0.293676,0.233860,0.472464
2,26459,0.090289,0.103902,0.805809
3,26460,0.122776,0.108186,0.769038
4,26461,0.064231,0.208494,0.727275


In [11]:
submit.to_csv('submit_catboost_20210408.csv',index=False)